In [2]:
import os
import cv2
import numpy as np
import torch
import tkinter as tk
from tkinter import filedialog, messagebox, colorchooser
from PIL import Image, ImageTk
from sklearn.cluster import KMeans
import torchvision
from torchvision.transforms import functional as F

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
OUTPUT_PATH = "result_offside_final.jpg"
OFFSIDE_LINES_OUTPUT_PATH = "offside_lines_result.jpg"

TEAM_COLORS = {
    0: (0, 0, 255),   # تیم 0: قرمز
    1: (255, 0, 0)    # تیم 1: آبی
}

model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(weights='DEFAULT')
model.eval().to(DEVICE)

TEXTS = {
    "fa": {
        "window_title": "رابط تشخیص آفساید - بخش کوچک زمین",
        "load_image": "بارگذاری تصویر",
        "select_image": "انتخاب تصویر فوتبال",
        "select_roi": "لطفاً ناحیه مورد نظر را انتخاب کنید",
        "detect_players": "تشخیص بازیکنان",
        "select_players": "انتخاب بازیکنان",
        "next": "بعدی",
        "back": "قبلی",
        "process": "پردازش نهایی",
        "attacking_team": "تیم حمله‌کننده (رنگ باکس):",
        "attack_direction": "جهت حمله:",
        "team": "تیم",
        "player": "بازیکن",
        "select": "انتخاب",
        "toggle_lang": "تغییر زبان",
        "detection_error": "هیچ بازیکنی تشخیص داده نشد.",
        "load_error": "ابتدا تصویری بارگذاری کنید.",
        "result_info": "نتیجه در فایل {} ذخیره شد.",
        "offside_info": "تصویر خط آفساید در فایل {} ذخیره شد.",
        "select_field_points": "انتخاب 4 نقطه‌ی مرجع (بخش کوچک زمین)",
        "undo_point": "(فشار دادن کلید 'u' برای حذف آخرین نقطه)",
        "offset_slider": "افست خط آفساید (جلو/عقب) [float]",
        "manual_offset": "ورود افست دستی (اعشاری):",
        "apply_offset": "اعمال افست",
        "line_thickness": "ضخامت خط:",
        "line_color": "رنگ خط:",
        "zoom_factor": "زوم تصویر:"
    },
    "en": {
        "window_title": "Offside Detection GUI - Partial Field",
        "load_image": "Load Image",
        "select_image": "Select Football Image",
        "select_roi": "Please select ROI",
        "detect_players": "Detect Players",
        "select_players": "Select Players",
        "next": "Next",
        "back": "Back",
        "process": "Process Final",
        "attacking_team": "Attacking Team (by box color):",
        "attack_direction": "Attack Direction:",
        "team": "Team",
        "player": "Player",
        "select": "Select",
        "toggle_lang": "Toggle Language",
        "detection_error": "No player detected.",
        "load_error": "Please load an image first.",
        "result_info": "Result saved in file {}.",
        "offside_info": "Offside line image saved in file {}.",
        "select_field_points": "Select 4 reference points (Partial Field)",
        "undo_point": "(Press 'u' to undo last point)",
        "offset_slider": "Offside Line Offset (forward/back) [float]",
        "manual_offset": "Manual Offset (float):",
        "apply_offset": "Apply",
        "line_thickness": "Line Thickness:",
        "line_color": "Line Color:",
        "zoom_factor": "Zoom Factor:"
    }
}


# --------------------------------------------------------------------------------
# تابع select_points که در گزارش خطا Missing بود، اینجاست.
def select_points(image, num_points=4, window_name="Select Points"):
    """
    این تابع به کاربر اجازه می‌دهد num_points نقطه را با کلیک ماوس انتخاب کند.
    کلید 'u' برای حذف آخرین نقطه انتخاب‌شده.
    کلید ESC برای لغو یا پایان.
    """
    clone = image.copy()
    pts = []

    def mouse_cb(event, x, y, flags, param):
        nonlocal pts, clone
        if event == cv2.EVENT_LBUTTONDOWN:
            pts.append((x, y))
            redraw()

    def redraw():
        nonlocal clone, pts
        clone = image.copy()
        for i,(px,py) in enumerate(pts):
            cv2.circle(clone, (px,py), 5, (0,0,255), -1)
            cv2.putText(clone, f"{i+1}", (px+5, py-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
        cv2.imshow(window_name, clone)

    cv2.namedWindow(window_name)
    cv2.setMouseCallback(window_name, mouse_cb)
    cv2.imshow(window_name, clone)

    while True:
        k = cv2.waitKey(50)&0xFF
        if k == 27:  # ESC
            break
        elif k == ord('u'):  # حذف آخرین نقطه
            if len(pts)>0:
                pts.pop()
                redraw()
        if len(pts)==num_points:
            break

    if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE)>=1:
        cv2.destroyWindow(window_name)
    return pts

# --------------------------------------------------------------------------------

def detect_players(image_bgr, confidence_threshold=0.6):
    # این تابع از قبل در سوال حاضر بود
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
    inp = F.to_tensor(image_rgb).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        out = model(inp)[0]
    players = []
    colors = []
    for idx, (box, lbl, score, mask) in enumerate(zip(
        out['boxes'].cpu().numpy(),
        out['labels'].cpu().numpy(),
        out['scores'].cpu().numpy(),
        out['masks'].cpu().numpy()
    )):
        if lbl == 1 and score > confidence_threshold:
            x1, y1, x2, y2 = map(int, box)
            roi = image_bgr[y1:y2, x1:x2]
            mask_roi = (mask[0]>0.5).astype(np.uint8)[y1:y2, x1:x2]
            if roi.size == 0 or mask_roi.sum()==0:
                continue
            dom_col = cv2.mean(roi, mask=mask_roi)[:3]
            players.append((idx, x1, y1, x2, y2, score))
            colors.append(dom_col)
    return players, colors


from sklearn.cluster import KMeans
def classify_teams(colors):
    # تابع خوشه‌بندی رنگ برای تعیین تیم
    if len(colors)<2:
        return [0]*len(colors)
    km = KMeans(n_clusters=2, random_state=42, n_init=10).fit(colors)
    labels = km.labels_
    ccs = km.cluster_centers_.flatten()
    if ccs[0]<ccs[1]:
        redc=0
        bluec=1
    else:
        redc=1
        bluec=0
    team_lbls=[]
    for l in labels:
        if l==redc:
            team_lbls.append(0)
        else:
            team_lbls.append(1)
    return team_lbls

def remove_goalkeepers(players, team_labels, w, h):
    # حذف بازیکنان گوشه‌ها (دروازه‌بان)
    gk_idxs=[]
    for i,(_,x1,y1,x2,y2,_) in enumerate(players):
        if min(x1,w-x2,y1,h-y2)< w*0.05:
            gk_idxs.append(i)
    new_pls=[p for i,p in enumerate(players) if i not in gk_idxs]
    new_lbls=[l for i,l in enumerate(team_labels) if i not in gk_idxs]
    return new_pls,new_lbls

def check_overlap(r1,r2):
    x1A,y1A,x2A,y2A = r1
    x1B,y1B,x2B,y2B = r2
    if x2A<x1B or x1A>x2B:
        return False
    if y2A<y1B or y1A>y2B:
        return False
    return True

def draw_label_anti_overlap(img, text, box, existing_rects):
    # جهت رسم لیبل‌ها بدون هم‌پوشانی
    font=cv2.FONT_HERSHEY_SIMPLEX
    fs=0.6
    thk=2
    x1,y1,x2,y2 = box
    (tw,th),_ = cv2.getTextSize(text,font,fs,thk)
    lx = x1
    ly = y1-5
    pad=4
    tries=0
    max_tries=30
    dy_step=10
    placed=False
    while tries<max_tries and not placed:
        rect=(lx, ly-th-pad, lx+tw+pad, ly)
        if rect[1]<0:
            ly+=dy_step
            tries+=1
            continue
        overlap=False
        for r in existing_rects:
            if check_overlap(rect,r):
                overlap=True
                break
        if overlap:
            ly-=dy_step
            tries+=1
        else:
            placed=True
            cv2.rectangle(img,(rect[0],rect[1]),(rect[2],rect[3]),(0,0,0),-1)
            cv2.putText(img,text,(rect[0]+2, rect[3]-2),font,fs,(255,255,255),thk)
            existing_rects.append(rect)
    if not placed:
        rect=(lx, ly-th-pad, lx+tw+pad, ly)
        cv2.rectangle(img,(rect[0],rect[1]),(rect[2],rect[3]),(0,0,0),-1)
        cv2.putText(img,text,(rect[0]+2, rect[3]-2),font,fs,(255,255,255),thk)
        existing_rects.append(rect)

def get_foot_center(player):
    _, x1,y1,x2,y2,_ = player
    return ((x1 + x2)//2, y2)

def warp_point(pt,H):
    arr=np.array([[[pt[0],pt[1]]]],dtype=np.float32)
    w=cv2.perspectiveTransform(arr,H)
    return (w[0,0,0], w[0,0,1])

def visualize_offside_line(image_bgr, players, team_labels, attacking_team, attack_direction,
                           H, H_inv, offset=0.0, line_thick=3, line_color=(0,255,0)):
    off_img = image_bgr.copy()
    defending_team = 1 - attacking_team
    defenders=[]
    for pl,lbl in zip(players, team_labels):
        if lbl==defending_team:
            foot = get_foot_center(pl)
            foot_top= warp_point(foot, H)
            defenders.append((pl, foot_top))
    if len(defenders)<2:
        return off_img
    if attack_direction=='left':
        defenders_sorted=sorted(defenders,key=lambda it: it[1][0], reverse=True)
    else:
        defenders_sorted=sorted(defenders,key=lambda it: it[1][0])
    sec_def, sec_def_top=defenders_sorted[1]
    x_def=sec_def_top[0]
    y_def=sec_def_top[1]
    topview_w=500
    y_line=y_def+ offset
    pt1=np.array([[[0, y_line]]],dtype=np.float32)
    pt2=np.array([[[topview_w, y_line]]],dtype=np.float32)

    pt1_img=cv2.perspectiveTransform(pt1, H_inv)[0,0]
    pt2_img=cv2.perspectiveTransform(pt2, H_inv)[0,0]
    x1,y1= int(pt1_img[0]), int(pt1_img[1])
    x2,y2= int(pt2_img[0]), int(pt2_img[1])

    cv2.line(off_img, (x1,y1),(x2,y2), line_color, int(line_thick))
    cv2.putText(off_img, f"Offset={offset:.2f}", (min(x1,x2)+5, min(y1,y2)+30),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,line_color,2)
    return off_img

# -----------------------------------------------------------
# رابط انتخاب بازیکنان
class SelectPlayersGUI:
    def __init__(self, parent, image_bgr, players, lang):
        self.parent=parent
        self.lang=lang
        self.top=tk.Toplevel(parent)
        self.top.title(TEXTS[self.lang]["select_players"])
        self.image_bgr=image_bgr
        self.players=players
        self.vars_select=[]

        main_frame=tk.Frame(self.top)
        main_frame.pack(fill="both",expand=True)

        # اسکرول کلی
        canvas=tk.Canvas(main_frame)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar=tk.Scrollbar(main_frame, orient="vertical", command=canvas.yview)
        scrollbar.pack(side="right", fill="y")
        canvas.configure(yscrollcommand=scrollbar.set)

        self.inner_frame=tk.Frame(canvas)
        canvas.create_window((0,0), window=self.inner_frame, anchor="nw")

        def on_configure(event):
            canvas.configure(scrollregion=canvas.bbox("all"))
        self.inner_frame.bind('<Configure>', on_configure)

        for i,(idx,x1,y1,x2,y2,score) in enumerate(players):
            fm=tk.Frame(self.inner_frame, bd=1, relief="solid")
            fm.pack(side="top", fill="x", padx=5, pady=5)
            roi=image_bgr[y1:y2, x1:x2].copy()
            cv2.rectangle(roi,(0,0),(roi.shape[1]-1,roi.shape[0]-1),(0,255,255),2)
            txt=f"{idx+1}"
            (tw,th),_ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX, 0.7,2)
            ccenter=(15,15)
            cv2.circle(roi, ccenter,15,(0,0,0),-1)
            cv2.putText(roi,txt,(ccenter[0]-tw//2, ccenter[1]+th//2),
                        cv2.FONT_HERSHEY_SIMPLEX,0.7,(255,255,255),2)
            roi_rgb=cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
            pil_img=Image.fromarray(roi_rgb)
            pil_img.thumbnail((80,80))
            tk_img=ImageTk.PhotoImage(pil_img)
            lbl_img=tk.Label(fm, image=tk_img)
            lbl_img.image=tk_img
            lbl_img.pack(side="left")

            lbl_info=tk.Label(fm, text=f"{TEXTS[self.lang]['player']} {idx+1} (Score={score:.2f})")
            lbl_info.pack(side="left", padx=5)

            var_select=tk.BooleanVar(value=False)
            chk_select=tk.Checkbutton(fm, text=TEXTS[self.lang]["select"],variable=var_select)
            chk_select.pack(side="left",padx=10)
            self.vars_select.append(var_select)

        btn_done=tk.Button(self.inner_frame, text=TEXTS[self.lang]["next"], command=self.on_done)
        btn_done.pack(pady=10)

        self.selected_indices=[]

    def on_done(self):
        self.selected_indices=[i for i,v in enumerate(self.vars_select) if v.get()]
        self.top.destroy()

def select_players_with_gui(parent, image_bgr, players, lang):
    gui=SelectPlayersGUI(parent, image_bgr, players, lang)
    parent.wait_window(gui.top)
    return gui.selected_indices

# ----------------------------------------------------------------
class OffsideGUI:
    def __init__(self, root):
        self.root=root
        self.lang="fa"
        self.root.title(TEXTS[self.lang]["window_title"])
        self.image_bgr=None
        self.players=[]
        self.colors=[]
        self.team_labels=[]
        self.attacking_team=0
        self.attack_direction='left'

        self.offside_offset=0.0
        self.offside_line_thickness=3
        self.offside_line_color=(0,255,0)
        self.zoom_factor=1.0

        self.H=None
        self.H_inv=None

        self.current_frame=None
        self.setup_stage1()

    def clear_frame(self):
        if self.current_frame:
            self.current_frame.destroy()

    # ------------ مرحله 1 ------------
    def setup_stage1(self):
        self.clear_frame()

        container = tk.Frame(self.root)
        container.pack(fill='both', expand=True)
        self.current_frame = container

        canvas = tk.Canvas(container)
        canvas.pack(side='left', fill='both', expand=True)
        sb = tk.Scrollbar(container, orient='vertical', command=canvas.yview)
        sb.pack(side='right', fill='y')
        canvas.configure(yscrollcommand=sb.set)

        frame_s1 = tk.Frame(canvas)
        canvas.create_window((0,0), window=frame_s1, anchor='nw')
        def on_conf(event):
            canvas.config(scrollregion=canvas.bbox('all'))
        frame_s1.bind('<Configure>', on_conf)

        # ویجت‌های اصلی مرحله 1
        btn_toggle = tk.Button(frame_s1, text="Toggle Lang", command=self.toggle_lang)
        btn_toggle.pack(pady=5)

        btn_load = tk.Button(frame_s1, text=TEXTS[self.lang]["load_image"], command=self.load_image)
        btn_load.pack(pady=5)

        btn_roi = tk.Button(frame_s1, text=TEXTS[self.lang]["select_roi"], command=self.select_roi)
        btn_roi.pack(pady=5)

        btn_detect = tk.Button(frame_s1, text=TEXTS[self.lang]["detect_players"], command=self.detect_players_action)
        btn_detect.pack(pady=5)

        self.lbl_image_stage1 = tk.Label(frame_s1)
        self.lbl_image_stage1.pack(padx=10, pady=10)

        self.btn_next_stage1 = tk.Button(frame_s1, text=TEXTS[self.lang]["next"],
                                         state="disabled", command=self.setup_stage1b)
        self.btn_next_stage1.pack(pady=5)

    def toggle_lang(self):
        self.lang = "en" if self.lang=="fa" else "fa"
        self.root.title(TEXTS[self.lang]["window_title"])
        self.setup_stage1()

    def load_image(self):
        ftypes=[("Image Files","*.jpg *.jpeg *.png *.webp")]
        fn = filedialog.askopenfilename(title=TEXTS[self.lang]["select_image"], filetypes=ftypes)
        if fn:
            self.image_bgr = cv2.imread(fn)
            self.show_stage1_image(self.image_bgr)

    def show_stage1_image(self, img):
        if img is None: return
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        pil_img.thumbnail((800,600))
        tkimg = ImageTk.PhotoImage(pil_img)
        self.lbl_image_stage1.config(image=tkimg)
        self.lbl_image_stage1.image = tkimg

    def select_roi(self):
        if self.image_bgr is None:
            messagebox.showerror(TEXTS[self.lang]["window_title"], TEXTS[self.lang]["load_error"])
            return
        roi_img,_=select_roi(self.image_bgr)
        self.image_bgr=roi_img
        self.show_stage1_image(self.image_bgr)

    def detect_players_action(self):
        if self.image_bgr is None:
            messagebox.showerror(TEXTS[self.lang]["window_title"], TEXTS[self.lang]["load_error"])
            return
        pls,cols=detect_players(self.image_bgr)
        if not pls:
            messagebox.showerror(TEXTS[self.lang]["window_title"], TEXTS[self.lang]["detection_error"])
            return
        self.players=pls
        self.colors=cols
        tmp=self.image_bgr.copy()
        rects=[]
        for (idx,x1,y1,x2,y2,_) in pls:
            cv2.rectangle(tmp,(x1,y1),(x2,y2),(0,255,255),2)
            draw_label_anti_overlap(tmp, f"{idx+1}", (x1,y1,x2,y2), rects)
        self.show_stage1_image(tmp)
        self.btn_next_stage1.config(state='normal')

    def setup_stage1b(self):
        sel= select_players_with_gui(self.root, self.image_bgr, self.players, self.lang)
        if sel:
            self.players=[self.players[i] for i in sel]
            self.colors=[self.colors[i] for i in sel]
        else:
            messagebox.showwarning(TEXTS[self.lang]["window_title"], "هیچ بازیکنی انتخاب نشد. ادامه با همه بازیکنان.")
        self.setup_stage2()

    # ------------ مرحله 2 ------------
    def setup_stage2(self):
        self.clear_frame()

        container = tk.Frame(self.root)
        container.pack(fill='both', expand=True)
        self.current_frame = container

        canvas = tk.Canvas(container)
        canvas.pack(side='left', fill='both', expand=True)
        sb = tk.Scrollbar(container, orient='vertical', command=canvas.yview)
        sb.pack(side='right', fill='y')
        canvas.configure(yscrollcommand=sb.set)

        frame_s2 = tk.Frame(canvas)
        canvas.create_window((0,0), window=frame_s2, anchor='nw')
        def on_conf(event):
            canvas.config(scrollregion=canvas.bbox('all'))
        frame_s2.bind('<Configure>', on_conf)

        h,w=self.image_bgr.shape[:2]
        self.team_labels=classify_teams(self.colors)
        self.players,self.team_labels=remove_goalkeepers(self.players,self.team_labels,w,h)

        tmp=self.image_bgr.copy()
        rects=[]
        for ((idx,x1,y1,x2,y2,_), lbl) in zip(self.players,self.team_labels):
            c=TEAM_COLORS[lbl]
            cv2.rectangle(tmp,(x1,y1),(x2,y2),c,2)
            draw_label_anti_overlap(tmp,f"{idx+1}", (x1,y1,x2,y2),rects)

        self.lbl_image_stage2 = tk.Label(frame_s2)
        self.lbl_image_stage2.pack(padx=10,pady=10)

        self.show_stage2_image(tmp)

        f_tm=tk.Frame(frame_s2)
        f_tm.pack(pady=5)
        tk.Label(f_tm, text=TEXTS[self.lang]["attacking_team"]).pack(side="left", padx=5)
        self.attacking_team_var=tk.IntVar(value=0)
        rb0=tk.Radiobutton(f_tm, text="0 (قرمز)", variable=self.attacking_team_var, value=0)
        rb1=tk.Radiobutton(f_tm, text="1 (آبی)", variable=self.attacking_team_var, value=1)
        rb0.pack(side="left", padx=5)
        rb1.pack(side="left", padx=5)

        f_dir=tk.Frame(frame_s2)
        f_dir.pack(pady=5)
        tk.Label(f_dir, text=TEXTS[self.lang]["attack_direction"]).pack(side="left", padx=5)
        self.attack_direction_var=tk.StringVar(value="left")
        rbL=tk.Radiobutton(f_dir, text="left", variable=self.attack_direction_var, value="left")
        rbR=tk.Radiobutton(f_dir, text="right", variable=self.attack_direction_var, value="right")
        rbL.pack(side="left", padx=5)
        rbR.pack(side="left", padx=5)

        fnav=tk.Frame(frame_s2)
        fnav.pack(pady=10)
        bback=tk.Button(fnav, text=TEXTS[self.lang]["back"], command=self.setup_stage1)
        bback.pack(side="left", padx=10)
        bnx=tk.Button(fnav, text=TEXTS[self.lang]["next"], command=self.setup_stage_homography)
        bnx.pack(side="left", padx=10)

    def show_stage2_image(self, img):
        if img is None: return
        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(rgb)
        pil_img.thumbnail((800,600))
        tkimg = ImageTk.PhotoImage(pil_img)
        self.lbl_image_stage2.config(image=tkimg)
        self.lbl_image_stage2.image = tkimg

    # ------------ مرحله 2.5 ------------
    def setup_stage_homography(self):
        self.clear_frame()

        container = tk.Frame(self.root)
        container.pack(fill='both', expand=True)
        self.current_frame = container

        canvas = tk.Canvas(container)
        canvas.pack(side='left', fill='both', expand=True)
        sb = tk.Scrollbar(container, orient='vertical', command=canvas.yview)
        sb.pack(side='right', fill='y')
        canvas.configure(yscrollcommand=sb.set)

        frame_h = tk.Frame(canvas)
        canvas.create_window((0,0), window=frame_h, anchor='nw')
        def on_conf(event):
            canvas.config(scrollregion=canvas.bbox('all'))
        frame_h.bind('<Configure>', on_conf)

        info_text= TEXTS[self.lang]["select_field_points"] + "\n" + TEXTS[self.lang]["undo_point"]
        lb=tk.Label(frame_h, text=info_text)
        lb.pack(pady=10)

        btn_sel=tk.Button(frame_h, text=TEXTS[self.lang]["select_field_points"], command=self.select_homography_points)
        btn_sel.pack(pady=5)

        fnav=tk.Frame(frame_h)
        fnav.pack(pady=10)
        bback=tk.Button(fnav, text=TEXTS[self.lang]["back"], command=self.setup_stage2)
        bback.pack(side="left", padx=10)
        bnx=tk.Button(fnav, text=TEXTS[self.lang]["next"], command=self.setup_stage3)
        bnx.pack(side="left", padx=10)

    def select_homography_points(self):
        if self.image_bgr is None:
            messagebox.showerror(TEXTS[self.lang]["window_title"], TEXTS[self.lang]["load_error"])
            return
        pts=select_points(self.image_bgr,4,"Select Partial Field Points")
        if len(pts)<4:
            messagebox.showerror(TEXTS[self.lang]["window_title"], "نقاط کافی انتخاب نشد.")
            return
        pts_src=np.array(pts,dtype=np.float32)
        pts_dst=np.array([
            [0,0],
            [500,0],
            [500,300],
            [0,300]
        ],dtype=np.float32)
        self.H=cv2.getPerspectiveTransform(pts_src, pts_dst)
        self.H_inv=cv2.getPerspectiveTransform(pts_dst, pts_src)
        messagebox.showinfo(TEXTS[self.lang]["window_title"], "چهار نقطه برای بخش کوچک ثبت شد.")

    # ------------ مرحله 3 ------------
    def setup_stage3(self):
        self.clear_frame()

        container=tk.Frame(self.root)
        container.pack(fill='both', expand=True)
        self.current_frame=container

        canvas=tk.Canvas(container)
        canvas.pack(side='left', fill='both', expand=True)
        sb=tk.Scrollbar(container, orient='vertical', command=canvas.yview)
        sb.pack(side='right', fill='y')
        canvas.configure(yscrollcommand=sb.set)

        frame_s3=tk.Frame(canvas)
        canvas.create_window((0,0), window=frame_s3, anchor='nw')
        def on_conf(event):
            canvas.config(scrollregion=canvas.bbox('all'))
        frame_s3.bind('<Configure>', on_conf)

        self.attacking_team=self.attacking_team_var.get()
        self.attack_direction=self.attack_direction_var.get()
        self.offside_offset=0.0

        if self.H is None or self.H_inv is None:
            messagebox.showwarning(TEXTS[self.lang]["window_title"],
                                   "کالیبره برای بخش کوچک انجام نشده!")
            self.offside_img=self.image_bgr.copy()
        else:
            self.offside_img= visualize_offside_line(
                self.image_bgr, self.players, self.team_labels,
                self.attacking_team, self.attack_direction,
                self.H, self.H_inv,
                offset=self.offside_offset,
                line_thick=self.offside_line_thickness,
                line_color=self.offside_line_color
            )

        self.lbl_offside=tk.Label(frame_s3)
        self.lbl_offside.pack(padx=10, pady=10)
        self.show_image(self.offside_img, self.lbl_offside)

        tk.Label(frame_s3, text=TEXTS[self.lang]["offset_slider"]).pack(pady=5)
        self.slider_offset = tk.Scale(frame_s3, from_=-400.0, to=400.0, orient="horizontal",
                               command=self.update_line_slider, resolution=0.5, length=300)
        self.slider_offset.set(0.0)
        self.slider_offset.pack(pady=5)

        f_manual=tk.Frame(frame_s3)
        f_manual.pack(pady=5)
        tk.Label(f_manual, text=TEXTS[self.lang]["manual_offset"]).pack(side="left", padx=5)
        self.entry_offset = tk.Entry(f_manual, width=8)
        self.entry_offset.pack(side="left", padx=5)
        btn_apply=tk.Button(f_manual, text=TEXTS[self.lang]["apply_offset"], command=self.update_line_entry)
        btn_apply.pack(side="left", padx=5)

        # اسلایدر ضخامت خط
        f_thick=tk.Frame(frame_s3)
        f_thick.pack(pady=5)
        tk.Label(f_thick, text=TEXTS[self.lang]["line_thickness"]).pack(side="left", padx=5)
        self.slider_thick=tk.Scale(f_thick, from_=1, to=10, orient="horizontal",
                                   command=self.update_line_thick, resolution=1, length=200)
        self.slider_thick.set(self.offside_line_thickness)
        self.slider_thick.pack(side="left", padx=5)

        # رنگ خط
        f_color=tk.Frame(frame_s3)
        f_color.pack(pady=5)
        tk.Label(f_color, text=TEXTS[self.lang]["line_color"]).pack(side="left", padx=5)
        self.color_options = {
            "Green": (0,255,0),
            "Red": (0,0,255),
            "Blue": (255,0,0),
            "Custom...": None
        }
        self.color_var=tk.StringVar(value="Green")
        color_menu=tk.OptionMenu(f_color, self.color_var, *self.color_options.keys(),
                                 command=self.update_line_color)
        color_menu.pack(side="left", padx=5)

        f_zoom=tk.Frame(frame_s3)
        f_zoom.pack(pady=5)
        tk.Label(f_zoom, text=TEXTS[self.lang]["zoom_factor"]).pack(side="left", padx=5)
        self.slider_zoom=tk.Scale(f_zoom, from_=0.5, to=3.0, orient="horizontal",
                                  resolution=0.1, length=200, command=self.update_zoom)
        self.slider_zoom.set(1.0)
        self.slider_zoom.pack(side="left", padx=5)

        fnav=tk.Frame(frame_s3)
        fnav.pack(pady=10)
        bback=tk.Button(fnav, text=TEXTS[self.lang]["back"], command=self.setup_stage2)
        bback.pack(side="left", padx=10)
        bf=tk.Button(fnav, text=TEXTS[self.lang]["process"], command=self.finish)
        bf.pack(side="left", padx=10)

    def update_line_slider(self, val):
        try:
            self.offside_offset=float(val)
        except ValueError:
            self.offside_offset=0.0
        self.entry_offset.delete(0, tk.END)
        self.entry_offset.insert(0, f"{self.offside_offset:.2f}")
        self.redraw_offside()

    def update_line_entry(self):
        val=self.entry_offset.get().strip()
        try:
            fval=float(val)
        except ValueError:
            messagebox.showerror(TEXTS[self.lang]["window_title"], f"مقدار '{val}' معتبر نیست!")
            return
        self.offside_offset=fval
        self.slider_offset.set(fval)
        self.redraw_offside()

    def update_line_thick(self, val):
        try:
            self.offside_line_thickness=int(float(val))
        except:
            self.offside_line_thickness=3
        self.redraw_offside()

    def update_line_color(self, selected):
        if selected=="Custom...":
            clr_code=colorchooser.askcolor(title="Select Line Color")
            if clr_code and clr_code[0]:
                r,g,b=clr_code[0]
                self.offside_line_color=(int(b), int(g), int(r))
        else:
            self.offside_line_color=self.color_options[selected]
        self.redraw_offside()

    def redraw_offside(self):
        if self.image_bgr is None or self.H is None or self.H_inv is None:
            return
        self.offside_img= visualize_offside_line(
            self.image_bgr, self.players, self.team_labels,
            self.attacking_team, self.attack_direction,
            self.H, self.H_inv,
            offset=self.offside_offset,
            line_thick=self.offside_line_thickness,
            line_color=self.offside_line_color
        )
        self.show_image(self.offside_img, self.lbl_offside)

    def update_zoom(self, val):
        try:
            self.zoom_factor=float(val)
        except:
            self.zoom_factor=1.0
        self.show_image(self.offside_img, self.lbl_offside)

    def show_image(self, img, lbl):
        if img is None:
            return
        factor=self.zoom_factor
        rgb=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_img=Image.fromarray(rgb)
        w,h=pil_img.size
        new_w=int(w*factor)
        new_h=int(h*factor)
        pil_resized=pil_img.resize((new_w,new_h), Image.Resampling.LANCZOS)
        tkimg=ImageTk.PhotoImage(pil_resized)
        lbl.config(image=tkimg)
        lbl.image=tkimg

    def clear_frame(self):
        if self.current_frame:
            self.current_frame.destroy()

    def finish(self):
        if self.image_bgr is not None:
            cv2.imwrite(OUTPUT_PATH, self.image_bgr)
        if hasattr(self, 'offside_img') and self.offside_img is not None:
            cv2.imwrite(OFFSIDE_LINES_OUTPUT_PATH, self.offside_img)
            messagebox.showinfo(TEXTS[self.lang]["window_title"],
                                TEXTS[self.lang]["offside_info"].format(OFFSIDE_LINES_OUTPUT_PATH))
        else:
            messagebox.showinfo(TEXTS[self.lang]["window_title"],
                                TEXTS[self.lang]["result_info"].format(OUTPUT_PATH))

if __name__=="__main__":
    root = tk.Tk()
    app = OffsideGUI(root)
    root.mainloop()
